In [1]:
import pandas as pd
from decouple import config
from neo4j import GraphDatabase
from dotenv import load_dotenv
import os

In [2]:
class InserirDadosNeo4j:
    MAX_NODES_LIMIT = 100000  # Defina o limite máximo de nós
    
    def __init__(self):
        # Carrega as informações do arquivo .env
        self.node_count = 0
        self.uri = config('NEO4J_URI')
        self.user = config('NEO4J_USERNAME')
        self.password = config('NEO4J_PASSWORD')
        self._driver = GraphDatabase.driver(self.uri, auth=(self.user, self.password))
    
    def close(self):
        self._driver.close()
        
    def insert_data_from_csv(self, csv_file):
        with self._driver.session() as session:
            df = pd.read_csv(csv_file)

            for index, row in df.iterrows():
                # Verifica se o número total de nós excede o limite
                if self.node_count >= self.MAX_NODES_LIMIT:
                    print(f"Limite de nós alcançado ({self.node_count} >= {self.MAX_NODES_LIMIT})."
                          " Interrompendo a inserção de dados.")
                    break
                
                # Inserir dados na tabela Região
                self._insert_regiao(session, row)

                # Inserir dados na tabela UF
                self._insert_uf(session, row)

                # Inserir dados na tabela Municípios
                self._insert_municipio(session, row)

                # Inserir dados na tabela Beneficiários
                self._insert_beneficiario(session, row)

                # Inserir dados na tabela IES
                self._insert_ies(session, row)

                # Inserir dados na tabela Cursos
                self._insert_cursos(session, row)

                # Inserir dados na tabela Bolsas
                self._insert_bolsas(session, row)
                
                # Atualiza o contador de nós
                self.node_count += 1
    
    @staticmethod
    def _insert_regiao(tx, row):
        query = """
        MERGE (r:Regiao {
            nome_regiao: $nome_regiao
        })
        """
        tx.run(query, nome_regiao=row['REGIAO_BENEFICIARIO'])

    @staticmethod
    def _insert_uf(tx, row):
        query = """
        MERGE (uf:UF {
            uf: $uf
        })
        """
        tx.run(query, uf=row['UF_BENEFICIARIO'])

    @staticmethod
    def _insert_municipio(tx, row):
        query = """
        MATCH (uf:UF {uf: $uf})
        MERGE (m:Municipio {
            nome_municipio: $nome_municipio,
            uf_id: id(uf)
        })
        """
        tx.run(query, uf=row['UF_BENEFICIARIO'], nome_municipio=row['MUNICIPIO_BENEFICIARIO'])

    @staticmethod
    def _insert_beneficiario(tx, row):
        query = """
        MATCH (m:Municipio {nome_municipio: $nome_municipio})
        MERGE (b:Beneficiario {
            cpf: $cpf,
            sexo: $sexo,
            raca: $raca,
            data_nascimento: $data_nascimento,
            deficiente_fisico: $deficiente_fisico,
            municipio_id: id(m)
        })
        """
        tx.run(query, cpf=row['CPF_BENEFICIARIO'], sexo=row['SEXO_BENEFICIARIO'], raca=row['RACA_BENEFICIARIO'], data_nascimento=row['DATA_NASCIMENTO'], deficiente_fisico=row['BENEFICIARIO_DEFICIENTE_FISICO'], nome_municipio=row['MUNICIPIO_BENEFICIARIO'])

    @staticmethod
    def _insert_ies(tx, row):
        query = """
        MATCH (m:Municipio {nome_municipio: $nome_municipio})
        MATCH (r:Regiao {nome_regiao: $nome_regiao})
        MERGE (ies:IES {
            codigo_emec: $codigo_emec,
            nome: $nome,
            municipio_id: id(m),
            regiao_id: id(r)
        })
        """
        tx.run(query, codigo_emec=row['CODIGO_EMEC_IES_BOLSA'], nome=row['NOME_IES_BOLSA'], nome_municipio=row['MUNICIPIO_BENEFICIARIO'], nome_regiao=row['REGIAO_BENEFICIARIO'])

    @staticmethod
    def _insert_cursos(tx, row):
        query = """
        MERGE (c:Curso {
            nome_curso: $nome_curso,
            turno: $turno
        })
        """
        tx.run(query, nome_curso=row['NOME_CURSO_BOLSA'], turno=row['NOME_TURNO_CURSO_BOLSA'])

    @staticmethod
    def _insert_bolsas(tx, row):
        query = """
        MATCH (ies:IES {codigo_emec: $codigo_emec})
        MATCH (c:Curso {nome_curso: $nome_curso, turno: $turno})
        MATCH (b:Beneficiario {cpf: $cpf})
        MERGE (bo:Bolsa {
            ano_concessao: $ano_concessao,
            tipo_bolsa: $tipo_bolsa,
            modalidade_ensino: $modalidade_ensino,
            ies_id: id(ies),
            curso_id: id(c),
            beneficiario_id: id(b)
        })
        """
        tx.run(query, codigo_emec=row['CODIGO_EMEC_IES_BOLSA'], nome_curso=row['NOME_CURSO_BOLSA'], turno=row['NOME_TURNO_CURSO_BOLSA'], cpf=row['CPF_BENEFICIARIO'], ano_concessao=row['ANO_CONCESSAO_BOLSA'], tipo_bolsa=row['TIPO_BOLSA'], modalidade_ensino=row['MODALIDADE_ENSINO_BOLSA'])


In [3]:
inserir_dados_neo4j = InserirDadosNeo4j()
inserir_dados_neo4j.insert_data_from_csv("../data/processed/ProuniRelatorio_processed.csv")
inserir_dados_neo4j.close()

Failed to read from defunct connection IPv4Address(('622dfb18.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.78.76.49', 7687)))


SessionExpired: Failed to read from defunct connection IPv4Address(('622dfb18.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.78.76.49', 7687)))